In [1]:
## this version is for the format: e.g. 1P22, 1endp22

import numpy as np
from math import floor
import music21
import re
import os

def get_file(txt):
    with open(txt, 'r') as file:
        string = file.read()
        return string

def arrToStreamChordwise(score, sample_freq, note_offset):
    # define variables and lists needed for chord decoding
    speed = 1./sample_freq
    piano_notes = []
    violin_notes = []
    time_offset = 0

    for i in range(len(score)):
        # skip empty lists
        if len(score[i]) == 0:
            continue

        # skip idx 0 as it indicates instruments
        for j in range(1, len(score[i])):

            # if the note is played, append note to list
            if score[i][j] == "1":
                duration=2

                # create music21 note object
                new_note = music21.note.Note(j + note_offset)

                # add duration to note object
                new_note.duration = music21.duration.Duration(duration * speed)

                # add timestamp (time offset) to note object
                new_note.offset = (i + time_offset) * speed


                # append note to respective instruments
                if score[i][0] == 'p':
                    piano_notes.append(new_note)
                elif score[i][0] == 'v':
                    violin_notes.append(new_note)

    # list of all notes for each instrument should be ready at this stage

    # creating music21 instrument objects
    violin = music21.instrument.fromString("Violin")
    piano = music21.instrument.fromString("Piano")

    # insert instrument object to start (0 index) of notes list
    violin_notes.insert(0, violin)
    piano_notes.insert(0, piano)

    # create music21 stream object for individual instruments
    violin_stream = music21.stream.Stream(violin_notes)
    piano_stream = music21.stream.Stream(piano_notes)

    # merge both stream objects into a single stream of 2 instruments
    chord_stream = music21.stream.Stream([violin_stream, piano_stream])
    #chord_stream.insert(0.0,music21.tempo.MetronomeMark('maestoso',tempo))
    
    return chord_stream

def arrToStreamNotewise(score, sample_freq, note_offset):    
    # define variables and lists needed for chord decoding
    speed = 1./sample_freq
    piano_notes = []
    violin_notes = []
    time_offset = 0
    
    i = 0
    while i < len(score):
        if score[i][0:9] == "p_octave_":
            add_wait=""
            if score[i][-3:] == "eoc":
                add_wait = "eoc"
                score[i] = score[i][:-3]
            this_note = score[i][9:]
            score[i] = "p"+this_note
            score.insert(i+1, "p" + str(int(this_note) + 12) + add_wait)
            i += 1
        i += 1

    for i in range(len(score)):
        if score[i] in ["", " ", "<eos>", "<unk>"]:
            continue
        elif score[i][0:3] == "end":
            if score[i][-3:] == "eoc":
                time_offset += 1
            continue
        elif score[i][:4] == "wait":
            time_offset += int(score[i][4:])
            continue
        else:
            # Look ahead to see if an end<noteid> was generated
            # soon after.  
            duration = 1
            has_end = False
            note_string_len = len(score[i])
            for j in range(1, 200):
                if i+j == len(score):
                    break
                if score[i+j][:4] == "wait":
                    duration += int(score[i+j][4:])
                if score[i+j][:3+note_string_len] == "end" + score[i] or score[i+j]==score[i]:
                    has_end = True
                    break
                if score[i+j][-3:] == "eoc":
                    duration += 1

            if not has_end:
                duration = 12

            add_wait = 0
            if score[i][-3:] == "eoc":
                score[i] = score[i][:-3]
                add_wait = 1

            try: 
                new_note = music21.note.Note(int(score[i][1:]) + note_offset)    
                new_note.duration = music21.duration.Duration(duration * speed)
                new_note.offset = time_offset*speed
                if score[i][0] == "v":
                    violin_notes.append(new_note)
                else:
                    piano_notes.append(new_note)                
            except:
                print("Unknown note: " + score[i])
                
            time_offset += add_wait


    # list of all notes for each instrument should be ready at this stage

    # creating music21 instrument objects      
    violin = music21.instrument.fromString("Violin")
    piano = music21.instrument.fromString("Piano")

    # insert instrument object to start (0 index) of notes list
    violin_notes.insert(0, violin)
    piano_notes.insert(0, piano)

    # create music21 stream object for individual instruments
    violin_stream = music21.stream.Stream(violin_notes)
    piano_stream = music21.stream.Stream(piano_notes)

    # merge both stream objects into a single stream of 2 instruments
    note_stream = music21.stream.Stream([violin_stream, piano_stream])

    # add tempo
    #note_stream.insert(0.0,music21.tempo.MetronomeMark('Larghetto', tempo))

    return note_stream

def write_midi(s, filename, output_folder):
    fp = s.write('midi', fp=output_folder+filename+".mid")
    
def decoding_music(notetxt,notename,sample_freq,note_offset):
    #chordstring = get_file(chordtxt)
    notestring = get_file(notetxt)
    
    #score_chord = chordstring.split(" ")
    score_note = notestring.split(" ")
    
    #chord_stream = arrToStreamChordwise(score_chord, sample_freq, note_offset, tempo)
    note_stream = arrToStreamNotewise(score_note, sample_freq, note_offset)

    pattern = re.compile('.*?/.*?/.*?/(.*?).txt')
    #chordname = re.findall(pattern, chordtxt)[0]
    notename = re.findall(pattern, notetxt)[0]
    
    #write_midi(chord_stream, chordname, chord_output)
    write_midi(note_stream, notename, note_output)
    

#note_output = "output_midi/Requiem/"
#notetxt = 'Requiem/lacrimosa_finished_20epoch.txt'
#sample_freq = 12
#note_offset = 33
#tempo = 80
#decoding_music(notetxt, note_output, sample_freq, note_offset, tempo)




In [2]:
import os
txtfiles = os.listdir('training_output/samples/mozart')
txtfiles

['result_epoch35_batch300.txt',
 'result_epoch36_batch500.txt',
 'result_epoch0_batch200.txt',
 'result_epoch3_batch800.txt',
 'result_epoch19_batch900.txt',
 'result_epoch35_batch0.txt',
 'result_epoch16_batch800.txt',
 'result_epoch39_batch0.txt',
 'result_epoch39_batch700.txt',
 'result_epoch6_batch300.txt',
 'result_epoch23_batch500.txt',
 'result_epoch20_batch500.txt',
 'result_epoch5_batch400.txt',
 'result_epoch13_batch800.txt',
 'result_epoch3_batch200.txt',
 'result_epoch39_batch400.txt',
 'result_epoch13_batch500.txt',
 'result_epoch19_batch400.txt',
 'result_epoch17_batch200.txt',
 'result_epoch36_batch300.txt',
 'result_epoch0_batch500.txt',
 'result_epoch8_batch300.txt',
 'result_epoch10_batch200.txt',
 'result_epoch9_batch200.txt',
 'result_epoch30_batch500.txt',
 'result_epoch18_batch300.txt',
 'result_epoch21_batch200.txt',
 'result_epoch3_batch500.txt',
 'result_epoch33_batch700.txt',
 'result_epoch27_batch400.txt',
 'result_epoch31_batch700.txt',
 'result_epoch35_batc

In [7]:
sample_freq = 12
note_offset = 33
#tempo = 80
txtfiles = os.listdir('training_output/samples/mozart')
note_output = "output_midi/mozart/"
for i in range(len(txtfiles)):
    if txtfiles[i] != '.ipynb_checkpoints':
        notetxt = 'training_output/samples/mozart/'+ txtfiles[i]
        decoding_music(notetxt, note_output, sample_freq, note_offset)
        
        